Apply point cloud completion (based on PoinTr) to a xyz cloud of any size. Most suitable for single trees.

In [ ]:
import numpy as np
import os
import glob
#import open3d as o3d
import tree2cubes


### Load point cloud

In [ ]:
# path to incomplete point cloud
infile = "path/to/incomplete/pointcloud"

In [ ]:
# load incomplete pointcloud
point_cloud = np.loadtxt(infile, skiprows=1, delimiter=" ") # delimiter=","

# load .ply file and convert to numpy array
# ply_cloud = o3d.io.read_point_cloud(item)
# point_cloud = np.asarray(ply_cloud.points)

### Cut into samples

PoinTr only allows input point clouds of a limited size. The treePoinTr models were trained on point cloud samples of 1m^3  containing between 2730 and 8192 points.
To apply completion on entire trees or even plots, larger point clouds need to be cut into cubes (voxels) to perform inference.
We use the function cut_point_cloud() to voxelize the point cloud four times with spatially shifted grids and specifyable voxel sizes. 

In [ ]:
# Cut the point cloud into cubes and save as .txt files. Choose 4 cube sizes approx. between 0.6 and 1.8 m
outpath = "path/to/cubes/"
tree2cubes.cut_point_cloud(point_cloud, outpath, size1=1, size2=1, size3=1.25, size4=1.8)

Optional data augmentation step:
make addtional versions of the cubes where x and z are switched. 
(inference results are sometimes rotation dependent)

In [ ]:
# make versions of the cubes where x and z are switched
path="path/to/cubes/"
for files in glob.glob(path+"*.txt"): 
    data = np.loadtxt(files)
    filename = os.path.basename(files)
    # Swap the first and third columns
    flipfile = np.column_stack((data[:, 2], data[:, 1], data[:, 0]))
    np.savetxt(path+filename+"_flip.txt", flipfile)

### Inference

Inference the samples with a pretrained model, following the instructions on https://github.com/yuxumin/PoinTr

For example, inference all samples under cubes/ and save the results under inference_result/, using the model finetune_groverealistic_newreal_noscale:


python tools/inference.py \
cfgs/groverealistic_models/PoinTr.yaml groverealistic_models/finetune_groverealistic_newreal_noscale/ckpt-best.pth \
--pc_root cubes/ \ 
--save_vis_img  \
--out_pc_root inference_result/ \


### Convert and merge predictions

In [ ]:
# convert all .npy files of predictions into .xyz (or .ply)
pred_path="/inference_result"
dirs = os.listdir(path=pred_path)
full_pred = np.empty((2, 3))
for dirs in dirs:
    a = np.load(pred_path+"/"+dirs+"/"+"fine.npy")
    np.savetxt(pred_path+"/"+dirs+"_pred.xyz", a)
    # cloud = o3d.geometry.PointCloud()
    # cloud.points = o3d.utility.Vector3dVector(a)
    # o3d.io.write_point_cloud(pred_path+"/"+dirs+"_pred.ply", cloud)
print("done")

In [ ]:
# reverse the switch of x and z on predictions if necessary
# and merge all predictions into one cloud  

full_pred = np.empty((2, 3))
pred1 = np.empty((2, 3))
predflip = np.empty((2, 3))


for files in glob.glob(pred_path+"*.xyz"): 
    data = np.loadtxt(files)
    filename = os.path.basename(files)
    newfile = np.column_stack((data[:, 2], data[:, 1], data[:, 0]))
    # Swap the first and third columns
    if "flip" in filename:
        #print("found flip")
        predflip = np.concatenate((predflip, newfile), 0)   
    else:
        newfile = data
        pred1 = np.concatenate((pred1, newfile), 0)
   

np.savetxt(pred_path+"/treename_completion.xyz", pred1)
np.savetxt(pred_path+"/treename_completion_withflips.xyz", predflip)
print("done")

### Post-processing

Ideally, the completed point clouds are now filtered in CloudCompare, using e.g. SOR filter and Gemetric features (Surface density).